### 時間序列新增  7 日 搭乘量 (7DaysTimes)
### 時間序列次數改為 移動平均 (MATimes)

In [1]:
import time
import pandas as pd
import datetime
import os

In [2]:
# 檔案讀取位置
datasite = './dfTime_TLC-2022.parquet'

# 是否輸出檔案 >0 輸出
df_save = 1
# 檔案輸出位置 
custom_output_path = f'./7DaysTimes'
DataName = '2022'

#設定 {N+M+1}DaysTimes 天數 
N = 3 #往前天數
M = 3 #往後天數

#-------------------------------------------------------------------------------------------------#
# 記錄開始時間
start_time = time.time()

df = pd.read_parquet(datasite)

# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time

print("程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())
df

程式執行花費的時間： 0.55 秒
done時間: 2023-08-26 23:49:39.891148


,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數
0,lyft,2022,1,1,0,5,3,True,16
1,lyft,2022,1,1,0,5,4,True,49
2,lyft,2022,1,1,0,5,5,True,1
3,lyft,2022,1,1,0,5,6,True,1
4,lyft,2022,1,1,0,5,7,True,115
...,...,...,...,...,...,...,...,...,...
5100013,yellow,2022,12,31,23,5,256,True,3
5100014,yellow,2022,12,31,23,5,259,True,1
5100015,yellow,2022,12,31,23,5,261,True,8
5100016,yellow,2022,12,31,23,5,262,True,35


### 新增 前N後M日搭乘量 (NDaysTimes)
    預設前3後3 總共7天

In [ ]:
# 記錄開始時間
start_time = time.time()

#創立一個 df_temp 統計N天內總次數
df['TempTime'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str)+'-' +df['day'].astype(str),
                                format='%Y-%m-%d')
df_temp = df[['Name','PULocationID','TempTime','次數']]
df_temp = df_temp.groupby([ 'Name','TempTime','PULocationID'])['次數'].sum().reset_index()

TempTime_unique_count = df['TempTime'].nunique()
print(f'總天數:{TempTime_unique_count}')
temp = 0
temp1 = '0'

for index, row in df_temp.iterrows():
    temp_start_time = row['TempTime'] - pd.DateOffset(days=N)
    temp_end_time = row['TempTime'] + pd.DateOffset(days=M)
    temp_count = df_temp.loc[(df_temp['TempTime'] >= temp_start_time) & (df_temp['TempTime'] <= temp_end_time) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['次數'].sum()
    temp_day =  row['TempTime'] - df_temp['TempTime'].min()
    if row['TempTime'].day == 1:  # 每個月的1日印出來
        if (temp != row['TempTime']) | (temp1  != row['Name']) :
            temp = row['TempTime']
            temp1 = row['Name']
            print(row['Name'],row['TempTime'])
    if TempTime_unique_count - temp_day.days <= M:          
        temp_count =  df_temp.loc[(df_temp['TempTime'] >= df_temp['TempTime'].max() -pd.DateOffset(days=N)-pd.DateOffset(days=M)) & (df_temp['TempTime'] <= df_temp['TempTime'].max() ) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['次數'].sum()
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
    elif temp_day.days >= N:  
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
    else : #處理前N-1天
        temp_count =  df_temp.loc[(df_temp['TempTime'] >= df_temp['TempTime'].min()) & (df_temp['TempTime'] <= ( df_temp['TempTime'].min() +pd.DateOffset(days=N)+pd.DateOffset(days=M)) ) & (row['Name'] == df_temp['Name']) & (row['PULocationID'] == df_temp['PULocationID']) ]['次數'].sum()
        #temp_count = temp_count *(N+1) / (temp_day.days+1)        
        df_temp.at[index,f'{N+1+M}DaysTimes'] = temp_count
        
del temp_start_time,temp_end_time,temp_count,temp_day,temp,temp1  

# 放回 df
#df[f'{N+1+M}DaysTimes'] = df.apply(lambda row:df_temp.loc[(df_temp['Name'] == row['Name']) &(df_temp['TempTime'] == row['TempTime']),f'{N+1+M}DaysTimes'].values[0], axis=1)
merged_df = df.merge(df_temp[['Name', 'TempTime','PULocationID', f'{N+1+M}DaysTimes']], on=['Name', 'TempTime','PULocationID'], how='left')
df[f'{N+1+M}DaysTimes'] = merged_df[f'{N+1+M}DaysTimes']
del merged_df

# TempTime移除
df = df.drop(columns=['TempTime'])
#df['TempTime'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str)+'-' +df['day'].astype(str) + ' ' + df['hour'].astype(str) ,format='%Y-%m-%d %H')

# 輸出檔案
if df_save > 0 :
    os.makedirs(os.path.dirname(f'{custom_output_path}/dfTime_TLC-{DataName}.parquet'), exist_ok=True)
    df.to_parquet(f'{custom_output_path}/dfTime_TLC-{DataName}.parquet')
    print('save!')
    
# 記錄結束時間
end_time = time.time()   
# 計算程式執行花費的時間
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())

總天數:365
lyft 2022-01-01 00:00:00
lyft 2022-02-01 00:00:00
lyft 2022-03-01 00:00:00
lyft 2022-04-01 00:00:00
lyft 2022-05-01 00:00:00
lyft 2022-06-01 00:00:00
lyft 2022-07-01 00:00:00
lyft 2022-08-01 00:00:00
lyft 2022-09-01 00:00:00
lyft 2022-10-01 00:00:00
lyft 2022-11-01 00:00:00
lyft 2022-12-01 00:00:00
uber 2022-01-01 00:00:00
uber 2022-02-01 00:00:00
uber 2022-03-01 00:00:00
uber 2022-04-01 00:00:00
uber 2022-05-01 00:00:00
uber 2022-06-01 00:00:00
uber 2022-07-01 00:00:00
uber 2022-08-01 00:00:00
uber 2022-09-01 00:00:00
uber 2022-10-01 00:00:00
uber 2022-11-01 00:00:00


In [22]:
df[df['PULocationID']==100].head(15)

,Name,year,month,day,hour,weekday,PULocationID,is_holiday,次數,7DaysTimes
92,lyft,2022,1,1,0,5,100,True,14,4.0
340,lyft,2022,1,1,1,5,100,True,20,4.0
586,lyft,2022,1,1,2,5,100,True,38,4.0
831,lyft,2022,1,1,3,5,100,True,27,4.0
1070,lyft,2022,1,1,4,5,100,True,39,4.0
1308,lyft,2022,1,1,5,5,100,True,18,4.0
1538,lyft,2022,1,1,6,5,100,True,9,4.0
1772,lyft,2022,1,1,7,5,100,True,11,4.0
2011,lyft,2022,1,1,8,5,100,True,9,4.0
2246,lyft,2022,1,1,9,5,100,True,7,4.0


In [23]:
#檢查用
df_temp[df_temp['PULocationID']==100].head(10)

,Name,TempTime,PULocationID,次數,7DaysTimes
98,lyft,2022-01-01,100,676,3549.0
353,lyft,2022-01-02,100,554,3549.0
610,lyft,2022-01-03,100,510,3549.0
865,lyft,2022-01-04,100,432,3549.0
1121,lyft,2022-01-05,100,382,3439.0
1377,lyft,2022-01-06,100,501,3413.0
1634,lyft,2022-01-07,100,494,3357.0
1891,lyft,2022-01-08,100,566,3420.0
2149,lyft,2022-01-09,100,528,3455.0
2405,lyft,2022-01-10,100,454,3477.0
